In [23]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.linalg import LinAlgError

from fooof import FOOOF, FOOOFGroup
from fooof.analysis import *
from fooof.synth import *

from utils.ratios import *
from utils.analysis import *

In [2]:
THETA_BAND = [4,8]
BETA_BAND = [15,25]

In [4]:
# Open CSV
data = pd.read_csv("./dat/MIPDB_PublicFile.csv")

## Young Age

In [5]:
young = data[(data['Age'] >= 10) & (data['Age'] <= 13)]

#Extract ID's to open their corresponding files
young_IDs = young.ID.values

#Open Eyes closed data for all IDs
problem_files = []
missing_files = []
young_params = []
young_tb_ratios = []

for filename in young_IDs:
    try:
        # Construct FOOOFGroup model on psd
        curr = np.load('./dat/psds/'+ filename + '_ec_psds.npz')
        fg = FOOOFGroup(verbose=False,)
        fg.add_data(curr['arr_0'], curr['arr_1'][0])
        fg.fit()
        
        # Following ratio is only for one block
        young_tb_ratios.append(calc_band_ratio(curr['arr_0'],curr['arr_1'][0], THETA_BAND, BETA_BAND))
        
        # Append params to respective array
        young_params.append(fg.get_all_data('background_params'))
        young_params.append(fg.get_all_data('peak_params'))
        
    except FileNotFoundError:
        missing_files.append(filename)
    except ValueError:
        problem_files.append(filename)
    except LinAlgError:
        problem_files.append(filename)

In [22]:
#test = np.load('./dat/psds/'+ filename + '_ec_psds.npz')
#test['arr_1'][0][0]
len(old_tb_ratios)

11

In [12]:
young_master = []
# aggregates the parameters for young people
for i in range(0,63,2):
    bgs = young_params[i].tolist()
    gausses = young_params[i+1].tolist()
    
    bgs_df = pd.DataFrame({"Background" : bgs})
    gausses_df = pd.DataFrame({"Gaussians" : gausses})
    
    cur = [bgs_df,gausses_df]
    young_master.append(cur)

In [ ]:
you

## Old Age

In [13]:
old = data[(data['Age'] >= 25)]

#Extract ID's to open their corresponding files
old_IDs = old.ID.values

#Open Eyes closed data for all IDs
problem_files_old = []
missing_files_old = []
old_params = []
old_tb_ratios = []
for filename in old_IDs:
    try:
        curr = np.load('./dat/psds/'+ filename + '_ec_psds.npz')
        fg = FOOOFGroup(verbose=False)
        fg.add_data(curr['arr_0'], curr['arr_1'][0])
        fg.fit()
        
        # Following ratio is only for one block
        old_tb_ratios.append(calc_band_ratio(curr['arr_0'], curr['arr_1'][0], THETA_BAND, BETA_BAND))
        
        old_params.append(fg.get_all_data('background_params'))
        old_params.append(fg.get_all_data('peak_params'))
        
    except FileNotFoundError:
        missing_files_old.append(filename)
    except ValueError:
        problem_files_old.append(filename)
    except LinAlgError:
        problem_files_old.append(filename)

In [14]:
old_master = []
for i in range(0,7,2):
    bgs = old_params[i].tolist()
    gausses = old_params[i+1].tolist()
    
    bgs_df = pd.DataFrame({"Background" : bgs})
    gausses_df = pd.DataFrame({"Gaussians" : gausses})
    
    cur = [bgs_df, gausses_df]
    old_master.append(cur)

## Differences in BG params  

In [155]:
#young_master
y_bg_1 = 0
y_bg_2 = 0
count = 0

for i in young_master:
    y_bg_1 += i[0].values[0][0][0]
    y_bg_2 += i[0].values[0][0][1]
    count+=1
young_bg_1 = y_bg_1/(count+1)
young_bg_2 = y_bg_2/(count+1)

young_bg = [young_bg_1,young_bg_2]

In [156]:
o_bg_1 = 0
o_bg_2 = 0
count = 0

for i in old_master:
    o_bg_1 += i[0].values[0][0][0]
    o_bg_2 += i[0].values[0][0][1]
    count+=1
old_bg_1 = o_bg_1/(count+1)
old_bg_2 = o_bg_2/(count+1)

old_bg = [old_bg_1,old_bg_2]

In [164]:
print("background param 1 difference:",abs(old_bg[0] - young_bg[0]))
print("background param 2 difference:",abs(old_bg[1] - young_bg[1]))

background param 1 difference: 0.8852820763497281
background param 2 difference: 0.7435350192201011


## Differences in BandRatio Measures

In [20]:
diff_tb_ratio = np.abs(np.mean(young_tb_ratios) - np.mean(old_tb_ratios))
print("Average young Theta/Beta:", np.mean(young_tb_ratios))
print("Average old Theta/Beta:", np.mean(old_tb_ratios))
print("Difference in average Theta/Beta ratio", diff_tb_ratio)

Average young Theta/Beta: 8.189531639873604
Average old Theta/Beta: 4.1547301345251855
Difference in average Theta/Beta ratio 4.034801505348418
